# Canadian Provinces and Territories into the Spatial Feature Registry

#### This code is in progress.  The code registers the Canadian Provinces and Territories into the Spatial Feature Registry (SFR within Data Distilleries GC2 instance) using the following workflow.  All data are retained from the source (unaltered), three registration fields are added (_id, reg_date, reg_source) and data are exported to a GeoJSON file.   The GeoJSON file is then uploaded to ScienceBase to document the final data as it is represented in the SFR.  Currently we are uploading data to the SFR using a manual process, with plans to automate this step in the future. 

#### General workflow involves:
     1: Retrieve Data From Source (ScienceBase Item: https://www.sciencebase.gov/catalog/item/5ab555c6e4b081f61ab78093)
     2: Create GeoDataFrame and identify native crs
     3: Define Variables needed throughout process
     4: Create new ScienceBase item to describe registration process
     5: Build and export GeoJSON representation of the data.  This process includes the addition of two registration fields that document information about registration (reg_source-> points to new SB item), and a registered uuid (_id).  
     6: Upload GeoJSON file to new ScienceBase item to document what was registered into SFR, along with additional information about when and how registration occured.  This process will likely change as we introduce a more systematic way of tracking prov.   During this step the user will upload data to GC2 as well (SFR schema).  Currently this process is done manually through the UI.
     

Code by: Daniel Wieferich (USGS)

Date: 20180330

In [1]:
#Import Needed Packages
import geopandas as gpd
import urllib.request as ur
import subprocess
import geojson
from sfr_load_utils import *

#### Step 1: Retrieve data from source

In [2]:
### Step 1: Retrieve Dataset from ScienceBase
#Canadian Province and Territories Dataset stored at https://www.sciencebase.gov/catalog/item/5ab555c6e4b081f61ab78093

#Define url of zipped shapefile download
downloadUrl ='https://www.sciencebase.gov/catalog/file/get/5ab555c6e4b081f61ab78093?f=__disk__4b%2F5e%2Fcd%2F4b5ecde7655133bb7453079564cf963d555aaaad'
#Download government unit file to local directory
ur.urlretrieve(downloadUrl, 'gpr_00b11a_e.zip')
#In working directory unzips file
subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + 'gpr_00b11a_e.zip' )

0

#### Step 2: Import shapefile into GeoDataFrame and identify native crs

In [3]:
#note the file name has one more '0' than the zip folder
file = 'gpr_000b11a_e.shp'
#Create GeoDataFrame from downloaded shapefile
df = gpd.read_file(file)

In [4]:
#Eventually will need a coded method to extract the epsg number (used as variable later), might be tricky given how this is returned
df.crs

{'init': 'epsg:4269'}

In [5]:
df.head()

,PREABBR,PRENAME,PRFABBR,PRFNAME,PRNAME,PRUID,geometry
0,B.C.,British Columbia,C.-B.,Colombie-Britannique,British Columbia / Colombie-Britannique,59,(POLYGON ((-123.5397725159999 48.3173484960000...
1,Que.,Quebec,Qc,Québec,Quebec / Québec,24,(POLYGON ((-74.63814234299991 44.9998372320000...
2,Nvt.,Nunavut,Nt,Nunavut,Nunavut,62,(POLYGON ((-79.99351691899994 51.2772504150000...
3,P.E.I.,Prince Edward Island,Î.-P.-É.,Île-du-Prince-Édouard,Prince Edward Island / Île-du-Prince-Édouard,11,(POLYGON ((-62.49301775499993 46.0251520690000...
4,Sask.,Saskatchewan,Sask.,Saskatchewan,Saskatchewan,47,POLYGON ((-102.0000000129999 59.99999999800008...


#### Step 3: Define Variables

In [6]:
#User Defined Variables
epsg = {'code':'4269'}
expected_geom_type = 'MultiPolygon'
outfile_name = 'canadian_territories_provinces'
source_sbitem = '5ab555c6e4b081f61ab78093'
list_tags = ['Jurisdictional Units','Area Beyond National Jurisdiction','BIS Spatial Feature Registry','Canada']
date = '2018-03-30'


#### Step 4: Create SB Item to describe SFR Registration 

In [7]:
#Build SB Item to house SFR GeoJSON File, including description of item.  
#This step outputs source_uri (uri to the new sb item that describes the data) to be included as registration information.

#Turns list of tags into json format accepted by SB
sb_tags = build_sb_tags(list_tags)
#Create SB session and log in
sb = sb_login()   
#Creates JSON needed to build and describe new SB item
item_info = sfr_item_info(sb,source_sbitem, sb_tags, date)
#Builds new SB item
new_item = build_new_sfr_sbitem(sb,item_info)
#URI of new SB item.  This is inserted into GEOJSON so we have a direct connection in SFR to documentation... this step may not
#be needed as we build prov capabilities.
source_uri = str(new_item['link']['url'])
print (source_uri)

username: dwieferich@usgs.gov
········
https://www.sciencebase.gov/catalog/item/5abea8cee4b081f61ac14e35


#### Step 5: Build and export GeoJSON representation of data.  Add registration id and source_uri (newly created SB item). Verify that the correct number of features were included in the GeoJSON dataset.

In [8]:
collection = df_to_geojson(df, epsg, source_uri, expected_geom_type)
print (verify_correct_count(collection, df))

#export_geojson(outfile_name, collection)
#Add file to SB Item

Correct number of features


In [9]:
file = export_geojson(outfile_name, collection)
outfile_zip = zip_geojson(outfile_name)

#### Step 6: Upload GeoJSON file to ScienceBase Item and also upload to GC2 using UI (make sure to specify UTF-8 encoding and MultiPolygon).

In [10]:
sb.upload_file_to_item(new_item, outfile_zip)

{'dates': [{'dateString': '2018-03-29',
   'label': 'Creation',
   'type': 'creation'},
  {'dateString': '2011-11-29',
   'label': 'Begin Position',
   'type': 'beginPosition'},
  {'dateString': '2011-11-29',
   'label': 'End Position',
   'type': 'endPosition'}],
 'distributionLinks': [{'files': [{'contentType': 'application/zip',
     'name': 'canadian_territories_provinces.zip',
     'size': 40813445,
     'title': None}],
   'name': 'SpatialFeatureR.zip',
   'rel': 'alternate',
   'title': 'Download Attached Files',
   'type': 'downloadLink',
   'typeLabel': 'Download Link',
   'uri': 'https://www.sciencebase.gov/catalog/file/get/5abea8cee4b081f61ac14e35'}],
 'files': [{'checksum': None,
   'contentEncoding': None,
   'contentType': 'application/zip',
   'dateUploaded': '2018-03-30T21:16:08Z',
   'downloadUri': 'https://www.sciencebase.gov/catalog/file/get/5abea8cee4b081f61ac14e35?f=__disk__67%2F16%2F64%2F671664a7e04b8b2207cad33bbd2cdc0c98f4c0d3',
   'imageHeight': None,
   'imageW

In [ ]:
#Currently the new SB item needs to have some additional information uploaded.  The UI can be used for this for now but in the future we will want to build as much as we can into this process.